# **Chapter 05. Create Dataset**

* 基本上, 任何能夠以 `4D tensor (batch × height × width × channel)` 的形式作為輸入的資料都可以應用於電腦視覺的訓練方式！

---
##### 影像:
* 影像解析度 與 運算／儲存／傳輸所需資源 的 trade-off, 建議 `選擇壓縮格式(例如JPEG)、較高閾值(95%+)、較低解析度(取決於任務精細程度)`
* 通常影像會有3個channel(RGB), 但有些影像會有4個channel(RGBA), 其中A是 `alpha(透明度)`
* 常規影像處理流程: `影像以壓縮字串的形式讀取, 轉換為 3D uint8 tensor, 再將 [0, 255] 的像素值轉換為 [0, 1] 的浮點數`
* 現代最常用的排序是 [height, width, channel] 的順序, 稱為 channel-last 表達法, 例如 TensorFlow 就是如此 (早期則是 channel-first)

##### 地理空間資料:
* 從地圖產生的地理空間資料, 通常有可以被視為頻道的`柵格頻帶(raster band)`, 也就是具有特定特徵的像素(或更大的網格)被標註
* 例如: 圖片中的河流所在的網格, 像素值會被設定為1; 或是有15個州的地圖, 會產生15個頻道, 每個頻道各自標出一個州所在的像素

#### 音訊(audio)與視訊(video):
* 音訊是 1D 信號, 視訊則是 3D。 通常會建議採用專為他們設計的 ML 技術, 但其實也能夠利用處理影像的 ML 方法來簡單處理他們
* 音訊方面, 可以用 time-domain 或 frequency-domain 的資料當作輸入, 或是也可以用 NLP 的方式來處理它
* 視訊方面, 可以用 Conv3D 或是 其與RNN的結合來處理

---
### 手動標記標籤
* 可以`將各個類別都建立資料夾`, 再把對應的圖片放到資料夾中; 或是利用單一`CSV檔記錄下所有資料的 URL + label`
* 但 當圖片有不只一項 label 時, 就只能使用後者的方法了
* 大規模資料需要標記時, 可以使用 Computer Vision Annotation Tool 這個軟體來達成, 他提供了一個很好用於標記的UI介面
* 或是當一張圖片會有多項 label 時, 也可以使用 Jupyter Notebook 的互動功能來標記 (python 的 multi-label-pigeon)

##### Python: multi-label-pigeon

In [ ]:
%%bash
mkdir flower_images
for filename in 100080576_f52e8ee070_n.jpg 10140303196_b88d3d6cec.jpg 10172379554_b296050f82_n.jpg; do
  gsutil cp gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/daisy/$filename flower_images
done

In [3]:
import glob
filenames = glob.glob('flower_images/*.jpg')
print(filenames)

from multi_label_pigeon import multi_label_annotate
from IPython.display import display, Image

annotations = multi_label_annotate(
    filenames,
    options={'flower':['daisy','tulip', 'rose'], 'color':['yellow','red', 'other'],'location':['indoors','outdoors']},
    display_fn=lambda filename: display(Image(filename))
    )

['flower_images/10140303196_b88d3d6cec.jpg', 'flower_images/10172379554_b296050f82_n.jpg', 'flower_images/100080576_f52e8ee070_n.jpg']


HTML(value='0 examples annotated, 4 examples left')

flower


color


location


Output()

In [4]:
print(annotations)

{'flower_images/10140303196_b88d3d6cec.jpg': {'flower': ['daisy', 'daisy'], 'color': ['yellow', 'yellow'], 'location': ['indoors', 'outdoors', 'outdoors']}, 'flower_images/10172379554_b296050f82_n.jpg': {'flower': ['tulip'], 'color': ['red'], 'location': ['outdoors']}, 'flower_images/100080576_f52e8ee070_n.jpg': {'flower': ['daisy'], 'color': ['yellow'], 'location': ['outdoors', 'indoors']}}


### 自動標記標籤
* `Noisy Student`: 
    * 先人工手動標記一部份影像, 再利用這些影像訓練出 teacher model 並讓他預測更多的標籤
    * 再將兩者的組合當作訓練資料, 並利用 dropout 與 data augmentation 去訓練出 student model
    * 把表現得更好的 student model 更新成為新的 teacher, 不斷迭代訓練下去; 人工也可以在這時手動標記那些被模型視為預測信心值不高的影像
* `Self-Supervised Learning`:
    * 有時候在取得圖片時, 還無法立即得知他的標籤 (例如: 診斷前一段時間拍攝的醫學影像, 下雨或打雷前的氣象圖...)
    * 所以, 很多時候拍攝的當下無法標記出的影像, 也可能是值得保留的！


---
### Bias (偏見)
* 